In [45]:
# this workbook generates the analysis table with number of positive genes, AUROC score, p-value, and name for each GO group.

In [46]:
import pandas as pd
import numpy as np
from scipy import sparse, stats

In [47]:
raw_go_annotation_df = pd.read_csv('./data/filtered_go_annotation_df.csv.gz', compression='gzip')
go_dictionary_df = pd.read_csv('./data/go_dictionary_df.csv')

In [48]:
go_annotation_df = raw_go_annotation_df.set_index('go_id')
go_dictionary_df = go_dictionary_df.set_index('go_id')

In [49]:
go_annotation_df

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
go_id,,,,,,,,,,,,,,,,,,,,,
GO:0000002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:2001279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:2001280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:2001286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
go_dictionary_df

,name
go_id,
GO:0000001,mitochondrion inheritance
GO:0000002,mitochondrial genome maintenance
GO:0000003,reproduction
GO:0000006,high-affinity zinc transmembrane transporter a...
GO:0000007,low-affinity zinc ion transmembrane transporte...
...,...
GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...
GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...
GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...


In [51]:
ranked_list_df = pd.read_csv('./data/ranked_list_df.csv')

In [52]:
ranked_list_df

,TRDD1,TRBD1,IGHD1-1,ATXN8,IGHD6-19,IGHD6-13,IGHD3-3,IGHD3-9,PPY2P,IGHD5-12,...,PCMTD1,LHX8,PCBP1,RPL5,GPX7,MRPL15,NR4A2,MMADHC,ELOVL2,CDK1
0,20491,20490,20489,20488,20487,20486,20485,20484,20483,20482,...,10,9,8,7,6,5,4,3,2,1


In [53]:
columns_only_in_ranked_list_df = set(ranked_list_df.columns) - set(go_annotation_df.columns)
ranked_list_df.drop(columns=columns_only_in_ranked_list_df, inplace=True)

In [54]:
columns_only_in_go_annotation_df = set(go_annotation_df.columns) - set(ranked_list_df.columns)
go_annotation_df.drop(columns=columns_only_in_go_annotation_df, inplace=True)

In [55]:
go_annotation_df = go_annotation_df.reindex(columns=ranked_list_df.columns)
go_annotation_df = go_annotation_df.T
analysis_df = ranked_list_df @ go_annotation_df

In [56]:
analysis_df

go_id,GO:0000002,GO:0000003,GO:0000012,GO:0000014,GO:0000018,GO:0000019,GO:0000027,GO:0000028,GO:0000030,GO:0000038,...,GO:2001259,GO:2001267,GO:2001268,GO:2001269,GO:2001270,GO:2001279,GO:2001280,GO:2001286,GO:2001293,GO:2001300
0,78592,161399,109607,90852,1250286,63485,174148,139766,228548,304273,...,571456,155971,66903,89068,51300,93274,72574,49573,37091,48887


In [57]:
analysis_df = analysis_df.rename(index={0: 'sum_of_ranks'})

In [58]:
analysis_df = analysis_df.T

In [59]:
analysis_df

,sum_of_ranks
go_id,
GO:0000002,78592
GO:0000003,161399
GO:0000012,109607
GO:0000014,90852
GO:0000018,1250286
...,...
GO:2001279,93274
GO:2001280,72574
GO:2001286,49573


In [60]:
def calculate_auroc(sum_of_ranks: int, n_total: int, n_pos: int, n_neg: int) -> float:
        if n_pos == 0:
            return 0
        else:
            average_rank = sum_of_ranks / n_pos
            min_average_rank = (n_pos + 1)/2
            shifted_average_rank = average_rank - min_average_rank
            auroc = shifted_average_rank / n_neg
            return auroc

In [61]:
def calculate_pval(n_pos: int, n_neg: int, auroc: float) -> float:   
    U = auroc * n_pos * n_neg
    Z = (np.abs(U -(n_pos * n_neg / 2))) / np.sqrt(n_pos * n_neg *(n_pos + n_neg + 1) / 12)
    p = stats.norm.sf(Z)
    return p

In [62]:
columns = ['n_total', 'n_pos', 'n_neg', 'auroc', 'pval', 'name']
for column in columns: analysis_df[column] = 0

In [63]:
for index, row in analysis_df.iterrows():
    n_total = go_annotation_df.shape[0]
    n_pos = sum(go_annotation_df[index])
    n_neg = n_total - n_pos
    analysis_df.loc[index, 'n_total'], analysis_df.loc[index, 'n_pos'], analysis_df.loc[index, 'n_neg'] = n_total, n_pos, n_neg
    analysis_df.loc[index, 'auroc'] = calculate_auroc(analysis_df.loc[index, 'sum_of_ranks'], n_total, n_pos, n_neg)
    analysis_df.loc[index, 'pval'] = calculate_pval(n_pos, n_neg, analysis_df.loc[index, 'auroc'])
    analysis_df.loc[index, 'name'] = go_dictionary_df.loc[index, 'name']

In [64]:
analysis_df

,sum_of_ranks,n_total,n_pos,n_neg,auroc,pval,name
go_id,,,,,,,
GO:0000002,78592,19484,11,19473,0.366596,0.062734,mitochondrial genome maintenance
GO:0000003,161399,19484,14,19470,0.591731,0.117316,reproduction
GO:0000012,109607,19484,10,19474,0.562555,0.246652,single strand break repair
GO:0000014,90852,19484,10,19474,0.466247,0.355826,single-stranded DNA endodeoxyribonuclease acti...
GO:0000018,1250286,19484,136,19348,0.471614,0.126578,regulation of DNA recombination
...,...,...,...,...,...,...,...
GO:2001279,93274,19484,12,19472,0.398846,0.112481,regulation of unsaturated fatty acid biosynthe...
GO:2001280,72574,19484,9,19475,0.413801,0.185240,positive regulation of unsaturated fatty acid ...
GO:2001286,49573,19484,6,19478,0.424000,0.259539,regulation of caveolin-mediated endocytosis


In [65]:
sorted_analysis_df = analysis_df.sort_values('auroc', ascending=False)
sorted_analysis_df = sorted_analysis_df.drop(['sum_of_ranks', 'n_total', 'n_neg'], axis=1)

In [66]:
sorted_analysis_df

,n_pos,auroc,pval,name
go_id,,,,
GO:0042101,149,0.943297,4.150241e-78,T cell receptor complex
GO:0042571,12,0.939182,6.879982e-08,"immunoglobulin complex, circulating"
GO:0002377,96,0.931034,1.548845e-48,immunoglobulin production
GO:0019814,172,0.930326,1.124584e-84,immunoglobulin complex
GO:0002440,98,0.930084,2.727475e-49,production of molecular mediator of immune res...
...,...,...,...,...
GO:0090425,5,0.106915,1.165905e-03,acinar cell differentiation
GO:0005784,5,0.106114,1.142091e-03,Sec61 translocon complex
GO:0071256,6,0.095347,2.984945e-04,translocon complex


In [69]:
sorted_analysis_df.to_csv('./auroc_sorted_analysis_df.csv')